In [15]:
import sys
import os

# Go up to project root, then into util
util_path = os.path.abspath(os.path.join(os.getcwd(), '..', 'util'))
print("Adding to sys.path:", util_path)

if util_path not in sys.path:
    sys.path.append(util_path)



Adding to sys.path: c:\Users\Kirsingh\OneDrive - Advanced Micro Devices Inc\Desktop\ScreenWhisper\util


In [16]:
from crewai import Agent, Crew, Task, LLM, Process
import os
import pyautogui
from crewai.tools import tool
from dotenv import load_dotenv
from pydantic import BaseModel, Field
from typing import List
from crew_tools import click_at, scroll, type_text_at, wait
from ultralytics import YOLO
from PIL import Image
import importlib
parent_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))  # Goes up one level from current directory
sys.path.append(parent_dir)
from util.utils import get_som_labeled_img, check_ocr_box, get_caption_model_processor, get_yolo_model
load_dotenv()

True

In [3]:
device = 'cuda'
model_path=r"C:\Users\Kirsingh\OneDrive - Advanced Micro Devices Inc\Desktop\ScreenWhisper\weights\icon_detect\model_v1_5.pt"
som_model = get_yolo_model(model_path)
#som_model.to(device)
print('model to {}'.format(device))


caption_model_processor = get_caption_model_processor(model_name="florence2", model_name_or_path=r"C:\Users\Kirsingh\OneDrive - Advanced Micro Devices Inc\Desktop\ScreenWhisper\weights\icon_caption_florence")

model to cuda


Florence2LanguageForConditionalGeneration has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.


In [ ]:
import time
time.sleep(5)
image = pyautogui.screenshot("screen.jpg")
image_path = "screen.jpg"
image = Image.open(image_path)
image_rgb = image.convert('RGB')
print('image size:', image.size)

box_overlay_ratio = max(image.size) / 3200
draw_bbox_config = {
    'text_scale': 0.8 * box_overlay_ratio,
    'text_thickness': max(int(2 * box_overlay_ratio), 1),
    'text_padding': max(int(3 * box_overlay_ratio), 1),
    'thickness': max(int(3 * box_overlay_ratio), 1),
}
BOX_TRESHOLD = 0.1


start = time.time()
ocr_bbox_rslt, is_goal_filtered = check_ocr_box(image_path, display_img = False, output_bb_format='xyxy', goal_filtering=None, easyocr_args={'paragraph': False, 'text_threshold':0.9}, use_paddleocr=True)
text, ocr_bbox = ocr_bbox_rslt
cur_time_ocr = time.time() 

dino_labled_img, label_coordinates, parsed_content_list = get_som_labeled_img(image_path, som_model, BOX_TRESHOLD = BOX_TRESHOLD, output_coord_in_ratio=True, ocr_bbox=ocr_bbox,draw_bbox_config=draw_bbox_config, caption_model_processor=caption_model_processor, ocr_text=text,use_local_semantics=True, iou_threshold=0.7, scale_img=False, batch_size=128)
cur_time_caption = time.time() 

screen_width = 1920
screen_height = 1200

print(parsed_content_list)
print("-------------------------------------------")
center_coords_and_content_str = "\n".join(
    f"Content: {item['content']} | type: {item['type']} | Center: ({int((item['bbox'][0] + item['bbox'][2]) / 2 * screen_width)}, {int((item['bbox'][1] + item['bbox'][3]) / 2 * screen_height)})"
    for item in parsed_content_list
)
print(center_coords_and_content_str)


In [ ]:
@tool
def take_screenshot():
    '''
    Useful to tell the screen information

    Args:
        None

    Output:
        Returns whats on the screen
    '''


    image = pyautogui.screenshot("screen.jpg")
    image_path = "screen.jpg"
    image = Image.open(image_path)
    image_rgb = image.convert('RGB')
    print('image size:', image.size)

    box_overlay_ratio = max(image.size) / 3200
    draw_bbox_config = {
        'text_scale': 0.8 * box_overlay_ratio,
        'text_thickness': max(int(2 * box_overlay_ratio), 1),
        'text_padding': max(int(3 * box_overlay_ratio), 1),
        'thickness': max(int(3 * box_overlay_ratio), 1),
    }
    BOX_TRESHOLD = 0.05

    import time
    start = time.time()
    ocr_bbox_rslt, is_goal_filtered = check_ocr_box(image_path, display_img = False, output_bb_format='xyxy', goal_filtering=None, easyocr_args={'paragraph': False, 'text_threshold':0.9}, use_paddleocr=True)
    text, ocr_bbox = ocr_bbox_rslt
    cur_time_ocr = time.time() 

    dino_labled_img, label_coordinates, parsed_content_list = get_som_labeled_img(image_path, som_model, BOX_TRESHOLD = BOX_TRESHOLD, output_coord_in_ratio=True, ocr_bbox=ocr_bbox,draw_bbox_config=draw_bbox_config, caption_model_processor=caption_model_processor, ocr_text=text,use_local_semantics=True, iou_threshold=0.7, scale_img=False, batch_size=128)
    cur_time_caption = time.time() 

    screen_width = 1920
    screen_height = 1200

    center_coords_and_content_str = "\n".join(
    f"Content: {item['content']} | type: {item['type']} | Center: ({int((item['bbox'][0] + item['bbox'][2]) / 2 * screen_width)}, {int((item['bbox'][1] + item['bbox'][3]) / 2 * screen_height)})"
    for item in parsed_content_list
    )
    print(center_coords_and_content_str)

    return f"The current screen contains these elements Content is the name of the element, type of element and coordinates to click the element {center_coords_and_content_str}"

    

In [7]:
tools=[click_at, type_text_at, wait, take_screenshot]

In [6]:
my_llm = LLM(
    model="groq/qwen-qwq-32b",
    temperature=0
    #api_key = "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"
)

In [8]:
my_llm = LLM(
    model="gemini/gemini-2.0-flash",
    temperature=0.9
    #api_key = "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"
)

In [9]:
step_reasoner = Agent(
    role = "Computer Use Agent",
    goal = "Your goal is to execute the user query by using the tools given and the screen state given query is - {query}",
    backstory = '''You are an expert in automating windows using user prompts. Having great experience in
    handling complex tasks and successfully executing the tasks.''',
    llm = my_llm,
    allow_delegation=False,
    tools=tools,
    verbose=True
)

LLM value is already an LLM object


In [12]:
task1 = Task(
    description=('''Your task is to automate the user given query very carefully and handling every edge cases too
    and after performing a tool call you have to take the screenshot of the screen to check and the screen
    will tell you about the icons not whether a window is opened or not but it provides accurate coordinates
    to click so after the action it can be said that the action is performed and based that  
    screen output reason and take next steps andyou will not assume any coordinates to click and if you 
    are not able to get the coordinates stop and '''),
    agent=step_reasoner,
    expected_output = "The output should include the steps you followed to achieve the task",
)

In [13]:
crew = Crew(
    agents=[step_reasoner],
    tasks = [task1],
    verbose=1,
)

[2025-04-27 10:29:37,595] [ WARNING] __init__.py:537 - Overriding of current TracerProvider is not allowed


In [17]:
import time

In [14]:
time.sleep(5)
result = crew.kickoff(inputs={"query": "open google chrome and search for virat kohli"})

# Agent: Computer Use Agent
## Task: Your task is to automate the user given query very carefully and handling every edge cases too
    and after performing a tool call you have to take the screenshot of the screen to check and the screen
    will tell you about the icons not whether a window is opened or not but it provides accurate coordinates
    to click so after the action it can be said that the action is performed and based that  
    screen output reason and take next steps andyou will not assume any coordinates to click and if you 
    are not able to get the coordinates stop and 
image size: (1920, 1200)

0: 800x1280 23 icons, 2367.4ms
Speed: 17.1ms preprocess, 2367.4ms inference, 2.0ms postprocess per image at shape (1, 3, 800, 1280)
len(filtered_boxes): 27 10
time to get parsed content: 80.2219889163971


# Agent: Computer Use Agent
## Thought: Okay, I understand my task. I need to open Google Chrome, search for "virat kohli", and then provide the steps I took. I will use t

[2025-04-27 10:32:36,318] [   ERROR] llm.py:277 - LiteLLM call failed: litellm.RateLimitError: litellm.RateLimitError: VertexAIException - {
  "error": {
    "code": 429,
    "message": "You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits.",
    "status": "RESOURCE_EXHAUSTED",
    "details": [
      {
        "@type": "type.googleapis.com/google.rpc.QuotaFailure",
        "violations": [
          {
            "quotaMetric": "generativelanguage.googleapis.com/generate_content_free_tier_requests",
            "quotaId": "GenerateRequestsPerMinutePerProjectPerModel-FreeTier",
            "quotaDimensions": {
              "model": "gemini-2.0-flash",
              "location": "global"
            },
            "quotaValue": "15"
          }
        ]
      },
      {
        "@type": "type.googleapis.com/google.rpc.Help",
        "links": [
          {
            "de



LiteLLM.Info: If you need to debug this error, use `litellm.set_verbose=True'.

 Received None or empty response from LLM call.


[2025-04-27 10:32:42,773] [   ERROR] llm.py:277 - LiteLLM call failed: litellm.RateLimitError: litellm.RateLimitError: VertexAIException - {
  "error": {
    "code": 429,
    "message": "You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits.",
    "status": "RESOURCE_EXHAUSTED",
    "details": [
      {
        "@type": "type.googleapis.com/google.rpc.QuotaFailure",
        "violations": [
          {
            "quotaMetric": "generativelanguage.googleapis.com/generate_content_free_tier_requests",
            "quotaId": "GenerateRequestsPerMinutePerProjectPerModel-FreeTier",
            "quotaDimensions": {
              "location": "global",
              "model": "gemini-2.0-flash"
            },
            "quotaValue": "15"
          }
        ]
      },
      {
        "@type": "type.googleapis.com/google.rpc.Help",
        "links": [
          {
            "de



LiteLLM.Info: If you need to debug this error, use `litellm.set_verbose=True'.



...

[2025-04-27 10:33:37,740] [   ERROR] llm.py:277 - LiteLLM call failed: litellm.RateLimitError: litellm.RateLimitError: VertexAIException - {
  "error": {
    "code": 429,
    "message": "You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits.",
    "status": "RESOURCE_EXHAUSTED",
    "details": [
      {
        "@type": "type.googleapis.com/google.rpc.QuotaFailure",
        "violations": [
          {
            "quotaMetric": "generativelanguage.googleapis.com/generate_content_free_tier_requests",
            "quotaId": "GenerateRequestsPerMinutePerProjectPerModel-FreeTier",
            "quotaDimensions": {
              "location": "global",
              "model": "gemini-2.0-flash"
            },
            "quotaValue": "15"
          }
        ]
      },
      {
        "@type": "type.googleapis.com/google.rpc.Help",
        "links": [
          {
            "de



LiteLLM.Info: If you need to debug this error, use `litellm.set_verbose=True'.



[2025-04-27 10:33:40,155] [   ERROR] llm.py:277 - LiteLLM call failed: litellm.RateLimitError: litellm.RateLimitError: VertexAIException - {
  "error": {
    "code": 429,
    "message": "You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits.",
    "status": "RESOURCE_EXHAUSTED",
    "details": [
      {
        "@type": "type.googleapis.com/google.rpc.QuotaFailure",
        "violations": [
          {
            "quotaMetric": "generativelanguage.googleapis.com/generate_content_free_tier_requests",
            "quotaId": "GenerateRequestsPerMinutePerProjectPerModel-FreeTier",
            "quotaDimensions": {
              "location": "global",
              "model": "gemini-2.0-flash"
            },
            "quotaValue": "15"
          }
        ]
      },
      {
        "@type": "type.googleapis.com/google.rpc.Help",
        "links": [
          {
            "de



LiteLLM.Info: If you need to debug this error, use `litellm.set_verbose=True'.



KeyboardInterrupt: 